In [ ]:
import datetime
import numpy as np
#import scipy as sp
#import uuid
import hashlib
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import os
import re
import math
from datetime import date
import ftplib
from ftplib import FTP
import gzip as gz
import collections
from collections import deque,OrderedDict,namedtuple
import sys
import cProfile
import multiprocessing
import inspect
#https://github.com/chaithemleib/intervaltree - don't use one that cannot support objects attached to intervals
import yaml
import datetime
import time
import ast
import codecs
import uuid
import json
# add liftover
from pyliftover import LiftOver
import hgvs.parser
import hgvs.dataproviders.uta
import hgvs.assemblymapper
import hgvs.projector
# internet
import urllib3
import certifi
import xmlrpc.client

In [ ]:
# Notebook display options
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)
pd.set_option('max_colwidth',1000)
%matplotlib inline
%autosave 0

In [ ]:
hdp = hgvs.dataproviders.uta.connect()
variantmapper = hgvs.variantmapper.VariantMapper(hdp)
am = hgvs.assemblymapper.AssemblyMapper(hdp,assembly_name='GRCh37', alt_aln_method='splign',replace_reference=True)
hp = hgvs.parser.Parser()
hn = hgvs.normalizer.Normalizer(hdp)
vr = hgvs.validator.Validator(hdp=hdp)

# genomic coords tester

In [ ]:
hgvs_g = 'NC_000007.13:g.36561662C>T'

# cDNA coord tester

In [ ]:
hgvs_c = 'NM_001637.3:c.1582G>A'

# parse genomic

In [ ]:
var_g = hp.parse_hgvs_variant(hgvs_g)

In [ ]:
var_g

In [ ]:
var_g.posedit.pos.start

In [ ]:
str(var_g)

# get transcripts of a gene

In [ ]:
transcripts = am.relevant_transcripts(var_g)

In [ ]:
sorted(transcripts)

# project a genomic variant onto a cDNA variant

In [ ]:
var_c = am.g_to_c(var_g, 'NM_001637.3')

In [ ]:
var_c

In [ ]:
var_c2 = am.g_to_c(var_g, 'NM_001177507.1')
var_c2

In [ ]:
var_c3 = am.g_to_c(var_g,'NM_001177506.1')
var_c3

In [ ]:
str(var_c)

In [ ]:
str(var_c2)

In [ ]:
str(var_c3)

In [ ]:
#dir(var_c3)

# validate a variant

In [ ]:
var_c3.validate

In [ ]:
try:
    # shift the NM_001177506.1 position +2 1582 to 1584 and see if this validates
    vr.validate( hp.parse_hgvs_variant('NM_001177506.1:c.1584G>A') )
    vr.validate( hp.parse_hgvs_variant('NM_001637.3:c.1582G>A') )
except hgvs.exceptions.HGVSError as e:
    print(e)

In [ ]:
var_p = am.c_to_p(var_c)

In [ ]:
var_p

In [ ]:
 str(var_p)

In [ ]:
var_p.posedit.uncertain = False

In [ ]:
str(var_p)

In [ ]:
 var_p.format(conf={"p_3_letter": False})

# Normalize a variant 

In [ ]:
hn.normalize(hp.parse_hgvs_variant('NM_001166478.1:c.35_36insT'))

In [ ]:
c1 = hp.parse_hgvs_variant('NM_001166478.1:c.31del')

In [ ]:
c1

In [ ]:
c1n = hn.normalize(c1)

In [ ]:
c1n

In [ ]:
g = am.c_to_g(c1)

In [ ]:
g

In [ ]:
gn=am.c_to_g(c1n)

In [ ]:
gn

In [ ]:
c2 = am.g_to_c(g, c1.ac)

In [ ]:
c2

In [ ]:
var_c1 = hgvsparser.parse_hgvs_variant('NM_001261456.1:c.1762A>G')

In [ ]:
pj = hgvs.projector.Projector(hdp=hdp,
                              alt_ac='NC_000001.10',
                              src_ac=var_c1.ac,
                              dst_ac='NM_001261457.1')

In [ ]:
pj.project_variant_forward(var_c1)

In [ ]:
hdp = hgvs.dataproviders.uta.connect()
variantmapper = hgvs.variantmapper.VariantMapper(hdp)
hgvsparser = hgvs.parser.Parser()

# project cDNA onto protein

In [ ]:
var_c1 = hgvsparser.parse_hgvs_variant('NM_001261456.1:c.1762A>G')
var_p1 = variantmapper.c_to_p(var_c1, None)
var_c1, var_p1

In [ ]:
var_g = variantmapper.c_to_g(var_c1,'NC_000001.10')
var_g

# get info manually on a variants within genes

In [ ]:
txs = hdp.get_tx_for_gene('LY9')
txs

In [ ]:
txs = hdp.get_tx_for_gene('ABCA4')
txs

In [ ]:
var_c2 = variantmapper.g_to_c(var_g,'NM_001261457.1',alt_aln_method='splign')
var_p2 = variantmapper.c_to_p(var_c2, None)
var_c2, var_p2

# Normalize a structural variant

In [ ]:
v = hp.parse_hgvs_variant("NM_001166478.1:c.30_31insT")
str(hn.normalize(v))

# DATA! : test this on a real data-set

In [ ]:
dt={'ALLELEID': int,'VariationID': int, 'SCV': str, 'Submitter': str, 'chrom': str, 'pos': int, 'ref': str, 'alt': str, 'CLNHGVS': object}
hgvs_df=pd.read_csv('/mnt/c/Users/mcgold/DATA/HGVStestFile.tsv',dtype=dt,sep="\t",header=0)

# put this together and check a file

In [ ]:
# make a structure to hold a validation round 
# validate the genomic, cDNA/s and protein/s
# validate the normalizations of genomic, cDNA and protein
attr=['ID','assembly','var_g','var_c','var_p','isNormalized_g','isNormalized_c',
      'isNormalized_p','isValid_g','isValid_c','isValid_p','transcripts','xrefs','timestamp']
Validation = namedtuple('variant',attr) 
n_attr=len(attr)
priorID=0
tests_dict={}
d=datetime.datetime.today().strftime('%Y-%m-%d')
date=str(d)
assembly='GRCh37'
for nth,i in enumerate(hgvs_df.index):
    ID=hgvs_df.loc[i,'ALLELEID']
    if ID != priorID:
        CLNHGVS_list=ast.literal_eval(hgvs_df.loc[i,'CLNHGVS'])
        record=Validation._make(['.']*n_attr)
        record._replace(ID=ID,timestamp=date)
        tests_dict[ID]=record
        tests_dict[ID]=tests_dict[ID]._replace(ID=ID)
        tests_dict[ID]=tests_dict[ID]._replace(assembly='GRCh37')
        tests_dict[ID]=tests_dict[ID]._replace(timestamp=date)
#    sys.stdout.write(str(ID) + "\t" + str(CLNHGVS_list) + "\n")
# genomic data
        for g in CLNHGVS_list:
#            print(nth,g)
            var_g='.'
            transcripts='.'
            varPlst=[]
            tests_dict[ID]=tests_dict[ID]._replace(var_g=g)
# validate the genomic variant 
            try:
                var_g=hp.parse_hgvs_variant(str(g))
                x=vr.validate( hp.parse_hgvs_variant(str(var_g)))
                tests_dict[ID]=tests_dict[ID]._replace(isValid_g=x)
            except:
                tests_dict[ID]=tests_dict[ID]._replace(isValid_g=None)
# check normalization
            try:
                x=hn.normalize(hp.parse_hgvs_variant(str(var_g)))
            except:
                x=None
            if str(x)==str(var_g):
                tests_dict[ID]=tests_dict[ID]._replace(isNormalized_g=True)
            elif (x is not None) and (str(x)!=str(var_g)):
                tests_dict[ID]=tests_dict[ID]._replace(isNormalized_g=False)
            else:
                tests_dict[ID]=tests_dict[ID]._replace(isNormalized_g=None)
# check valid
            try:
                x=vr.validate(hp.parse_hgvs_variant(str(var_c)))
                tests_dict[ID]=tests_dict[ID]._replace(isValid_c=x)
            except:
                tests_dict[ID]=tests_dict[ID]._replace(isValid_c=None)
            try:
                var_p=variantmapper.c_to_p(var_c, None)
                x=vr.validate( hp.parse_hgvs_variant(str(var_p)))
                tests_dict[ID]=tests_dict[ID]._replace(isValid_p=x)
            except:
                tests_dict[ID]=tests_dict[ID]._replace(isValid_p=None)
# get the transcripts and check valid and normalized
            try:
                transcripts = sorted(am.relevant_transcripts(var_g))
                transcriptStr=";".join(transcripts)
                tests_dict[ID]=tests_dict[ID]._replace(transcripts=transcriptStr)
                sys.stdout.write(str(ID) + "\t" + str(CLNHGVS_list) + "\t" + str(transcripts) + "\n")
            except:
                tests_dict[ID]=tests_dict[ID]._replace(transcripts=None)
            try:
                tests_dict[ID]=tests_dict[ID]._replace(var_p=varPstr)
                xrefStr=transcriptStr + ';' + varPstr
                tests_dict[ID]=tests_dict[ID]._replace(xrefs=xrefStr)
            except:
                pass            
            for t in transcripts:
                if t!='.':
                    var_c=am.g_to_c(var_g, str(t))
                    var_p=variantmapper.c_to_p(var_c, None)
                    varPlst.append(str(var_p))
                    try:
                        x=hn.normalize(hp.parse_hgvs_variant(str(var_c)))
                    except:
                        x=None
                    if str(x)==str(var_c):
                         sys.stdout.write("\tNORMALIZATION PASS: " +  str(x) +"=" + str(var_c) + "\n")
                    elif (x is not None) and (str(x)!=str(var_c)):
                        sys.stderr.write("\tNORMALIZATION FAIL: " +  str(x) +"=" + str(var_c) + "\n")
                    else:
                        sys.stdout.write("\tNORMALIZATION NULL: " +  str(x) +"=" + str(var_c) + "\n")
        priorID=ID

In [ ]:
tests_dict

In [ ]:
pd.DataFrame.from_dict(tests_dict,orient='index').to_csv('/mnt/c/Users/mcgold/DATA/HGVSValidationResult.tsv',sep="\t",header=True,index=False)

In [ ]:
df=pd.DataFrame.from_dict(tests_dict,orient='index')

In [ ]:
df